In [ ]:
# SARIMAX (Seasonal AutoRegressive Integrated Moving-Average with eXogenous regressors) Model
# When to use SARIMAX:

# Seasonal Data: SARIMAX is appropriate for time series data that exhibits seasonality.
#     It extends the ARIMA model to capture seasonal effects with seasonal AR, I, and MA
#     components.
# Multivariate Time Series: SARIMAX can incorporate exogenous variable
#     s (additional predictors) which can improve forecasting accuracy if there are 
#     external factors influencing the time series.
# Complex Structures: When your data exhibits both seasonal patterns and non-seasonal
#     patterns, SARIMAX can capture these complexities.

In [ ]:
# ARIMA (AutoRegressive Integrated Moving Average) Model
# When to use ARIMA:

# Non-Seasonal Data: ARIMA is suitable for time series data that does not exhibit
#     seasonality. It can handle trends and non-stationary data by differencing.
# Univariate Time Series: ARIMA models are designed for univariate time series, where 
#     you are modeling and forecasting a single variable.
# Simple Structures: When your data can be well-modeled with just AR (autoregressive) 
#     terms, I (integrated, or differencing) terms, and MA (moving average) terms.

In [ ]:
# dataset link 
# https://github.com/viniciusov/candy-production/blob/master/candy_production.csv

In [ ]:
import pandas as pd
candy_prod_dataset = pd.read_csv('./data/candy_production.csv')
candy_prod_dataset.head()

In [ ]:
candy_prod_dataset['observation_date'] = pd.to_datetime(candy_prod_dataset['observation_date'],
                                             infer_datetime_format=True)
candy_prod_dataset.head()

In [ ]:
indexed_candy_prod_dataset = candy_prod_dataset.set_index(['observation_date'])
indexed_candy_prod_dataset = indexed_candy_prod_dataset['IPG3113N']
indexed_candy_prod_dataset.head()

In [ ]:
indexed_candy_prod_dataset.info()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose 
decompose_data = seasonal_decompose(indexed_candy_prod_dataset,model="additive")
decompose_data.plot()

In [ ]:
def test_stationarity(timeseries):
    import matplotlib.pyplot as plt 
    timeseries.dropna(inplace=True)
    rolmean = timeseries.rolling(window=12).mean()
    rolstd = timeseries.rolling(window=12).std()

    orig = plt.plot(timeseries,label='Orignal')
    mean = plt.plot(rolmean,label='Roling mean')
    std = plt.plot(rolstd,label='rolling std')
    plt.legend(loc='best')
    plt.show()
    from statsmodels.tsa.stattools import adfuller
    dftest = adfuller(timeseries)
    print("1. ADF : ",dftest[0])
    print("2. P_value  : ",dftest[1])
    print("3. Num od Lags : ",dftest[2])
    print("4. Num od observations used for ADF regression and critical values calculation : ",dftest[3])
    print("5. Critical values : ") 
    for key, val in dftest[4].items():
        print("\t",key, ": ",val)
#            .)p value is 8 so its not statinary .  

In [ ]:
test_stationarity(indexed_candy_prod_dataset)

In [ ]:
import numpy as np 
indexed_candy_prod_dataset_log_scaled = np.log(indexed_candy_prod_dataset)
test_stationarity(indexed_candy_prod_dataset_log_scaled)

In [ ]:
ma = indexed_candy_prod_dataset_log_scaled.rolling(window=12).mean()
indexed_candy_prod_dataset_log_scaled_minus_ma = indexed_candy_prod_dataset_log_scaled - ma

In [ ]:
test_stationarity(indexed_candy_prod_dataset_log_scaled_minus_ma)

In [ ]:
from statsmodels.tsa.stattools import acf,pacf 
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
pacf = pacf(indexed_candy_prod_dataset_log_scaled_minus_ma,nlags=140)
acf = acf(indexed_candy_prod_dataset_log_scaled_minus_ma,nlags=70)
plot_acf(acf)
plot_pacf(pacf)

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
indexed_candy_prod_dataset_log_scaled_minus_ma.dropna(inplace=True)
train,test = train_test_split(indexed_candy_prod_dataset_log_scaled_minus_ma,
                              test_size=0.10,shuffle=False)

model_arima=sm.tsa.arima.ARIMA(train,order=(1,1,2))
model_arima=model_arima.fit()
prediction = model_arima.predict(start ='2013-03-01',end='2017-08-01')

In [ ]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
error = mean_squared_error(test,prediction)
print('Test MSE %.3f' %error)
# pacf,acf prediction of p,q plot.
predict = np.exp(prediction)
test_set = np.exp(test)
plt.plot(test_set)
plt.plot(predict,color='red')
plt.show()

In [ ]:
# finding p and q by custon hit and trial.
import itertools
p = range(0,8)
q= range(0,8)
d = range(0,2)
pdq_combinations = list(itertools.product(p,d,q))
rmse=[]
order1 = []


In [ ]:
# import statsmodels.api as sm
# from sklearn.metrics import mean_squared_error
# for pdq in pdq_combinations:
#     model_arima=sm.tsa.arima.ARIMA(train,order=pdq)
#     model_arima=model_arima.fit()
#     prediction = model_arima.predict(start ='2013-03-01',end='2017-08-01')
#     error = np.sqrt(mean_squared_error(test,prediction))
#     order1.append(pdq)
#     rmse.append(error)

In [ ]:
results = pd.DataFrame(index=order1,data=rmse,columns=['RMSE'])

In [ ]:
results.head()

In [ ]:
# pqd_value = results[results['RMSE'] == results['RMSE'].min()]
# pqd_value = pqd_value.index[0]

In [ ]:
# pqd_value

In [ ]:
model_arima=sm.tsa.arima.ARIMA(train,order=(7,0,3))
model_arima=model_arima.fit()
prediction = model_arima.predict(start ='2013-03-01',end='2017-08-01')
from sklearn.metrics import mean_squared_error
error = mean_squared_error(test,prediction)
print('Test MSE %.3f' %error)

predict = np.exp(prediction)
test_set = np.exp(test)
plt.plot(test_set)
plt.plot(predict,color='red')
plt.show()

In [ ]:
model_sarimx=sm.tsa.statespace.SARIMAX(train,order=(7,0,3),seasonal_order=(7,0,3,12))
model_sarimx=model_sarimx.fit()
prediction = model_sarimx.predict(start ='2013-03-01',end='2017-08-01')

In [ ]:
error = mean_squared_error(test,prediction)
print('Test MSE %.3f' %error)
# pacf,acf prediction of p,q plot.
predict = np.exp(prediction)
test_set = np.exp(test)
plt.plot(test_set)
plt.plot(predict,color='red')
plt.show()